In [2]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re
from keras_preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [50]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/hotel_data.csv')

In [51]:
data.head()

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,...,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id
0,Hardasji Ki Magri,Udaipur,India,2016-06-21,{{facility}},|Zion Home Stay is located in a city that sets...,1 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,78ddf880bd7937d384ff278cc5b39d6e
1,Near Nai Gaon,Udaipur,India,2016-06-21,{{facility}},| Araliayas Resorts is a 3 star hotel located ...,3 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,9f9f9cbb2f7df8089b63d5cdeb257944
2,Near Bagore Ki Haveli,Udaipur,India,2016-06-21,{{facility}},|A 2 star property is located at 24 km from Ma...,2 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,NaN,makemytrip,NaN,NaN,b314bb7fa8bfb1ed306f517be21d729e
3,Dabok,Udaipur,India,2016-06-21,Airport Transfer|Car rental|Conference Hall|Cu...,|SNP House Airport Hotel And Restaurant is loa...,1 star,//imghtlak.mmtcdn.com/images/hotels/2014071815...,NaN,no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,`standard,/5,NaN,makemytrip,NaN,NaN,e6f5bb3c2d76a78d978b9ceb0e31ec56
4,East Udaipur,Udaipur,India,2016-06-21,{{facility}},| Hotel Pichola Haveli is situated in the beau...,2 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,3.7,makemytrip,NaN,NaN,63072c301427b6ca450d31eea127bcf0


In [52]:
data.city.value_counts()

NewDelhiAndNCR    1163
Goa               1122
Mumbai             543
Jaipur             534
Bangalore          512
                  ... 
Kollur               1
Madla                1
Jeypore              1
Jispa                1
Haldia               1
Name: city, Length: 770, dtype: int64

In [53]:
data.shape

(20000, 33)

In [54]:
array=['Mumbai','Goa']
data=data.loc[data['city'].isin(array)]

In [55]:
data.head()

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,...,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id
282,South Goa,Goa,India,2016-08-28,Bar|Doctor on Call|Extra Mattress (On Request)...,"Just 2.7 km from Margao Railway Station, Hotel...",2,NaN,Television|Air Conditioning|Attached Bathroom|...,yes,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Standard Ac,NaN,NaN,makemytrip,Goa,Location:4.3/5 | Hospitality:4.4/5 | Facilitie...,afa2b33d49d47b83026b94d54449192c
288,NaN,Goa,India,2016-08-28,Airport Transfer|Bicycles on Hire|Coffee Shop|...,"Offering swimming pool, lawns/gardens and wate...",3,NaN,Tea/Coffee Maker|Bathroom Toiletries|DVD Playe...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Luxury Flat 1 Bhk,NaN,NaN,makemytrip,Goa,NaN,4953f7d9c9a00e8246fcd58f0cf7d350
294,Charai,Mumbai,India,2016-08-28,Doctor on Call|Front desk|Laundry Service|Park...,"Nestled in Mumbai, a city with strong historic...",3,NaN,Bathroom Toiletries|Attached Bathroom|Hot & Co...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Ac Superior Room|A/c Standard Rooms Double Occ...,NaN,NaN,makemytrip,Maharashtra,NaN,d78fae90ef2e1b5c2dfd547c61763a25
296,NaN,Goa,India,2016-08-28,24 hour security|A/C Power Backup Available|Ai...,Diakon Holidays is situated in one of the most...,1,NaN,Telephone|Bathroom Toiletries|Gas Stove|Washin...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,4 Bedroom Pool Villa,NaN,NaN,makemytrip,Goa,NaN,2c57c7604510c1a2ab519a84a31cf030
297,Benaulim,Goa,India,2016-08-28,NaN,"Nestle in Goa, a land of never ceasing festivi...",1,NaN,NaN,no,...,Apartment,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,2 Bhk Apartment,NaN,NaN,makemytrip,Goa,NaN,248ec7a4718e571cdaa54009140e7fdc


In [56]:
data=data.hotel_overview

In [57]:
data.isna().sum()

169

In [58]:
data.dropna(inplace=True)
data.isna().sum()

0

In [59]:
data.iloc[0]

'Just 2.7 km from Margao Railway Station, Hotel Om Shiv offers easy access to the Colva Beach and has a restaurant and internet cafe. Located in the heart of Margao, close to the famous Colva Beach, Hotel Om Shiv is a popular budget property in this region of Goa. With all the necessary amenities and well-furnished rooms, the hotel promises to provide a comfortable stay to guests. There are three categories of rooms available at the hotel, including Executive Room, Deluxe Room and Suite. All rooms are appointed with basic amenities, including air-conditioner, cable television and attached bathroom with hot and cold water supply. This six storey property is an ideal base for both business and leisure travellers as it lies in the vicinity of business centres, government offices and entertainment areas of the city. Besides a rooftop party lounge, the hotel also houses a mini internet cafe and a well-equipped conference hall Diamond having a seating capacity of around 50 persons. Shivam is

In [60]:
stop=set(stopwords.words('english'))
print(stop)

{'them', 'only', 'am', 'my', 'those', 'once', 'herself', 'ain', 'of', 'your', 'into', 'be', 'such', 'their', "mustn't", 'about', 'against', 'isn', 'have', 'haven', 'no', 'to', 'who', 'we', 'what', 'while', 'his', 'too', 'were', 'being', 'it', 'here', "couldn't", 'down', 'than', 'yourselves', "aren't", 'should', 'for', 'she', 'me', 'doing', 'don', 'ma', 're', "shouldn't", "doesn't", 'by', 'which', "wouldn't", "should've", 'd', 'but', 'any', 'where', 'is', 'with', 'won', 'they', 'her', 'he', 'did', 'why', 'few', "you'll", 'as', 'between', 'll', "wasn't", 'ours', "didn't", "she's", 'or', 'the', "hasn't", 'whom', 'most', 'there', 'at', "needn't", 'through', 'so', 'its', 'over', 'and', 'again', 'further', 'doesn', 'o', 'yours', 'ourselves', 'now', 'couldn', 'him', 'needn', 'itself', 'before', 'very', 'himself', "you'd", 'a', 'aren', 'from', 'nor', 'when', "it's", 'below', 'not', "hadn't", 'during', 'i', "you've", 'wouldn', "haven't", 'was', 'does', 'both', 'how', 'more', 'yourself', 'until'

In [61]:
data.head()

282    Just 2.7 km from Margao Railway Station, Hotel...
288    Offering swimming pool, lawns/gardens and wate...
294    Nestled in Mumbai, a city with strong historic...
296    Diakon Holidays is situated in one of the most...
297    Nestle in Goa, a land of never ceasing festivi...
Name: hotel_overview, dtype: object

In [62]:
def stopwords_removal(data_point):
    data=[x for x in data_point.split() if x not in stop]
    return data

In [63]:
def clean_data(data):
    cleaned_data=[]
    all_unique_words_in_each_description=[]
    for entry in data:
        entry = re.sub(pattern='[^a-zA-Z]',repl=' ',string = entry)
        entry = re.sub(r'\b\w{0,1}\b', repl=' ',string = entry)
        entry = entry.lower()
        entry = stopwords_removal(entry)
        cleaned_data.append(entry)
        unique = list(set(entry))
        all_unique_words_in_each_description.extend(unique)
    return cleaned_data, all_unique_words_in_each_description

In [64]:
def unique_words(data):
    unique_words = set(data)
    return unique_words, len(unique_words)

In [65]:
cleaned_data, all_unique_words_in_each_description = clean_data(data)
unique_words, length_of_unique_words = unique_words(all_unique_words_in_each_description)

In [66]:
print(cleaned_data[0])

['km', 'margao', 'railway', 'station', 'hotel', 'om', 'shiv', 'offers', 'easy', 'access', 'colva', 'beach', 'restaurant', 'internet', 'cafe', 'located', 'heart', 'margao', 'close', 'famous', 'colva', 'beach', 'hotel', 'om', 'shiv', 'popular', 'budget', 'property', 'region', 'goa', 'necessary', 'amenities', 'well', 'furnished', 'rooms', 'hotel', 'promises', 'provide', 'comfortable', 'stay', 'guests', 'three', 'categories', 'rooms', 'available', 'hotel', 'including', 'executive', 'room', 'deluxe', 'room', 'suite', 'rooms', 'appointed', 'basic', 'amenities', 'including', 'air', 'conditioner', 'cable', 'television', 'attached', 'bathroom', 'hot', 'cold', 'water', 'supply', 'six', 'storey', 'property', 'ideal', 'base', 'business', 'leisure', 'travellers', 'lies', 'vicinity', 'business', 'centres', 'government', 'offices', 'entertainment', 'areas', 'city', 'besides', 'rooftop', 'party', 'lounge', 'hotel', 'also', 'houses', 'mini', 'internet', 'cafe', 'well', 'equipped', 'conference', 'hall',

In [67]:
length_of_unique_words

6151

In [68]:
def build_indices(unique_words):
    word_to_idx = {}
    idx_to_word = {}
    for i, word in enumerate(unique_words):
        word_to_idx[word] = i
        idx_to_word[i] = word
    return word_to_idx, idx_to_word

In [69]:
word_to_idx, idx_to_word = build_indices(unique_words)
print(idx_to_word)

{0: 'triceps', 1: 'commotions', 2: 'khargar', 3: 'mosquito', 4: 'ttc', 5: 'cook', 6: 'kolem', 7: 'tourist', 8: 'rahi', 9: 'bose', 10: 'burgerpao', 11: 'inns', 12: 'changers', 13: 'manor', 14: 'banyan', 15: 'msrtc', 16: 'pinto', 17: 'groups', 18: 'ecotel', 19: 'wired', 20: 'sundowner', 21: 'palate', 22: 'suitable', 23: 'grill', 24: 'walled', 25: 'persons', 26: 'intricately', 27: 'lanes', 28: 'river', 29: 'treatments', 30: 'andrews', 31: 'vacations', 32: 'economical', 33: 'international', 34: 'magnificently', 35: 'supported', 36: 'organising', 37: 'aspect', 38: 'carved', 39: 'siesta', 40: 'often', 41: 'kept', 42: 'house', 43: 'nicknamed', 44: 'carbon', 45: 'portico', 46: 'myriad', 47: 'industry', 48: 'viceroy', 49: 'aid', 50: 'mobor', 51: 'snow', 52: 'surcharge', 53: 'might', 54: 'energizing', 55: 'zinhos', 56: 'cul', 57: 'requirements', 58: 'derives', 59: 'gagal', 60: 'dial', 61: 'original', 62: 'day', 63: 'generous', 64: 'ganesha', 65: 'beauties', 66: 'lonavala', 67: 'mallikarjuna', 68

In [70]:
def prepare_corpus(corpus, word_to_idx):
    
    sequences = []
    for line in corpus:
        tokens = line
        for i in range(1, len(tokens)):
            i_gram_sequence = tokens[:i+1]
            i_gram_sequence_ids = []
            
            for j, token in enumerate(i_gram_sequence):
                i_gram_sequence_ids.append(word_to_idx[token])
                
            sequences.append(i_gram_sequence_ids)
    
    return sequences

In [71]:
sequences = prepare_corpus(cleaned_data, word_to_idx)
max_sequence_len = max([len(x) for x in sequences])

In [72]:
max_sequence_len

454

In [73]:
print(sequences[0])
print(sequences[1])

[3937, 4517]
[3937, 4517, 2485]


In [74]:
print(idx_to_word[1647])
print(idx_to_word[867])
print(idx_to_word[1452])

younger
finish
turtle


In [75]:
len(sequences)

165507

In [76]:
def build_input_data(sequences, max_sequence_len, length_of_unique_words):
    sequences = np.array(pad_sequences(sequences, maxlen = max_sequence_len, padding = 'pre'))
    X = sequences[:,:-1]
    y = sequences[:,-1]
    y = np_utils.to_categorical(y, length_of_unique_words)
    return X, y

In [77]:
length_of_unique_words

6151

In [78]:
X, y = build_input_data(sequences, max_sequence_len, length_of_unique_words)

In [79]:
def create_model(max_sequence_len, length_of_unique_words):
    model = Sequential()
    model.add(Embedding(length_of_unique_words, 10, input_length=max_sequence_len - 1))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(length_of_unique_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [80]:
model = create_model(max_sequence_len, length_of_unique_words)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 453, 10)           61510     
                                                                 
 lstm_1 (LSTM)               (None, 128)               71168     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 6151)              793479    
                                                                 
Total params: 926,157
Trainable params: 926,157
Non-trainable params: 0
_________________________________________________________________


In [81]:
model.fit(X, y, batch_size = 512, epochs=100)

Epoch 1/100
324/324 [==============================] - 54s 157ms/step - loss: 6.8987
Epoch 2/100
324/324 [==============================] - 34s 105ms/step - loss: 6.5938
Epoch 3/100
324/324 [==============================] - 29s 90ms/step - loss: 6.4037
Epoch 4/100
324/324 [==============================] - 28s 85ms/step - loss: 6.1209
Epoch 5/100
324/324 [==============================] - 26s 81ms/step - loss: 5.9137
Epoch 6/100
324/324 [==============================] - 24s 74ms/step - loss: 5.7023
Epoch 7/100
324/324 [==============================] - 24s 74ms/step - loss: 5.5093
Epoch 8/100
324/324 [==============================] - 24s 73ms/step - loss: 5.3067
Epoch 9/100
324/324 [==============================] - 24s 74ms/step - loss: 5.1285
Epoch 10/100
324/324 [==============================] - 23s 72ms/step - loss: 4.9669
Epoch 11/100
324/324 [==============================] - 23s 70ms/step - loss: 4.8173
Epoch 12/100
324/324 [==============================] - 23s 71ms/step - 

In [82]:
model_structure = model.to_json()
with open("/content/drive/MyDrive/Datasets/model/text_generation_using_LSTM_Mumbai_Goa.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights("/content/drive/MyDrive/Datasets/model/text_generation_using_LSTM_Mumbai_Goa.h5")

In [85]:
from keras.models import model_from_json
json_file=open('/content/drive/MyDrive/Datasets/model/text_generation_using_LSTM_Mumbai_Goa.json','r')
loaded_model_json=json_file.read()
json_file.close()
loaded_model=model_from_json(loaded_model_json)
loaded_model.load_weights("/content/drive/MyDrive/Datasets/model/text_generation_using_LSTM_Mumbai_Goa.h5")
print('Model Loaded from Drive')

Model Loaded from Drive


In [45]:
def generate_text(seed_text, next_words, model, max_seq_len):
    for _ in range(next_words):
        cleaned_data = clean_data([seed_text])
        sequences= prepare_corpus(cleaned_data[0], word_to_idx)
        sequences = pad_sequences([sequences[-1]], maxlen=max_seq_len-1, padding='pre')
        predicted_x = model.predict(sequences, verbose=0)
        predicted=np.argmax(predicted_x,axis=1)
        output_word = ''
        output_word = idx_to_word[predicted[0]]            
        seed_text = seed_text + " " + output_word
        
    return seed_text.title()

In [91]:
print(generate_text("Destination Goa", 100, loaded_model, max_sequence_len))

Destination Goa Located City Beautiful Beaches Goa Located Distance Km Goa International Airport Km Karmali Railway Station Km Mapusa Bus Stand Hotel Close Proximity Various Tourist Places Like Shree Siddhivinayak Temple Km Mount Mary Church Km Shree Siddhivinayak Temple Km Mount Mary Church Km Shree Siddhivinayak Temple Km Many Others Situated Feasible Distance Hotel Offers Facilities Like Front Desk Parking Laundry Service Parking Facility Also Offers Facilities Like Taxi Service Airport Railway Station Transfer Taxi Service Airport Railway Station Transfer Robert Place Leisure Travellers Hotel Offers Facilities Like Parking Facility Doctor Call Facility Also Available Chargeable Basis Hotel Kumkum Features Well Appointed


In [47]:
print(generate_text("Best Mumbai", 30, model, max_sequence_len))

Best Mumbai Apartments Located Heart City Mumbai Mumbai Nearest Airport Chhatrapati Shivaji International Airport Km Away Bhayandar Railway Station Mumbai Central Railway Station Km Away Bus Stand Popular Tourist Attractions Like Shree


In [ ]:
!git add TextGenerationLSTM.ipynb
!git commit -m "Text Generation Using LSTM just tried with 10 epochs but very less efficient"
!git push

[main ff1ce07] Text Generation Using LSTM just tried with 10 epochs but very less efficient
 1 file changed, 1254 insertions(+)
 create mode 100644 TextGenerationLSTM.ipynb
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 6 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 31.98 KiB | 7.99 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To github.com:Aayush181509/NLPLearning.git
   6aa09d1..ff1ce07  main -> main
